In [1]:
import sys
!{sys.executable} -m pip install numpy
!{sys.executable} -m pip install matplotlib
!{sys.executable} -m pip install torch
!{sys.executable} -m pip install pandas
!{sys.executable} -m pip install sklearn
!{sys.executable} -m pip install tqdm
!{sys.executable} -m pip install scipy
!{sys.executable} -m pip install tensorboardX
!{sys.executable} -m pip install imbalanced-learn
!{sys.executable} -m pip install h5py
!{sys.executable} -m pip install tensorflow
!{sys.executable} -m pip install keras
!{sys.executable} -m pip install stable-baselines3
!{sys.executable} -m pip install cloudpickle
!{sys.executable} -m pip install streamlit
!{sys.executable} -m pip install pyngrok
!{sys.executable} -m pip install seaborn
!{sys.executable} -m pip install tf-explain
!{sys.executable} -m pip install xgboost
!{sys.executable} -m pip install mlflow
!{sys.executable} -m pip install shap
!{sys.executable} -m pip install keras-tuner
!{sys.executable} -m pip install scikit-optimize
!{sys.executable} -m pip install catboost
!{sys.executable} -m pip install scipy
!{sys.executable} -m pip install optuna
!{sys.executable} -m pip install keras_tuner
!{sys.executable} -m pip install tensorflow-decision-forests

  Using cached nvidia_cuda_nvrtc_cu12-12.1.105-py3-none-manylinux1_x86_64.whl.metadata (1.5 kB)
  Using cached nvidia_cuda_runtime_cu12-12.1.105-py3-none-manylinux1_x86_64.whl.metadata (1.5 kB)
  Using cached nvidia_cuda_cupti_cu12-12.1.105-py3-none-manylinux1_x86_64.whl.metadata (1.6 kB)
  Using cached nvidia_cudnn_cu12-8.9.2.26-py3-none-manylinux1_x86_64.whl.metadata (1.6 kB)
  Using cached nvidia_cublas_cu12-12.1.3.1-py3-none-manylinux1_x86_64.whl.metadata (1.5 kB)
  Using cached nvidia_cufft_cu12-11.0.2.54-py3-none-manylinux1_x86_64.whl.metadata (1.5 kB)
  Using cached nvidia_curand_cu12-10.3.2.106-py3-none-manylinux1_x86_64.whl.metadata (1.5 kB)
  Using cached nvidia_cusolver_cu12-11.4.5.107-py3-none-manylinux1_x86_64.whl.metadata (1.6 kB)
  Using cached nvidia_cusparse_cu12-12.1.0.106-py3-none-manylinux1_x86_64.whl.metadata (1.6 kB)
  Using cached nvidia_nccl_cu12-2.20.5-py3-none-manylinux2014_x86_64.whl.metadata (1.8 kB)
  Using cached nvidia_nvtx_cu12-12.1.105-py3-none-manylinu

In [2]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import StandardScaler, PolynomialFeatures, LabelEncoder, OneHotEncoder
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier, StackingClassifier, VotingClassifier
from sklearn.model_selection import StratifiedKFold, cross_val_score, GridSearchCV, train_test_split, KFold, RandomizedSearchCV, cross_val_predict
from sklearn.neighbors import KNeighborsClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, confusion_matrix, classification_report, roc_auc_score, roc_curve, auc
from imblearn.over_sampling import SMOTE
import matplotlib.pyplot as plt
import seaborn as sns
import xgboost as xgb
from xgboost import XGBClassifier
from lightgbm import LGBMClassifier
from catboost import CatBoostClassifier
from skopt import BayesSearchCV
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.impute import SimpleImputer
from sklearn.svm import SVC
from sklearn.tree import DecisionTreeClassifier
from sklearn.neural_network import MLPClassifier
from imblearn.over_sampling import SMOTE
import scipy.stats as stats
from scipy.stats import uniform, randint
from scipy.stats import randint as sp_randint
from scipy.stats import uniform as sp_uniform
import re
import shap
import optuna
from optuna.samplers import TPESampler
import tensorflow as tf
import keras_tuner as kt
import tensorflow_decision_forests as tfdf
from tensorflow.keras import layers, Model
from imblearn.over_sampling import ADASYN

from google.colab import drive
import torch
import warnings
import logging
warnings.filterwarnings('ignore')


/usr/local/lib/python3.10/dist-packages/dask/dataframe/__init__.py:42: FutureWarning: 
Dask dataframe query planning is disabled because dask-expr is not installed.

You can install it with `pip install dask[dataframe]` or `conda install dask`.
This will raise in a future version.

  warnings.warn(msg, FutureWarning)


In [3]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print('Using device:', device)


Using device: cpu


In [4]:
drive.mount('/content/drive')

! cd drive


Mounted at /content/drive


In [5]:
# Set up logging
logging.basicConfig(level=logging.INFO, format='%(asctime)s - %(levelname)s - %(message)s')

# Initialize logging
logger = logging.getLogger(__name__)

# Set up logging to also write to a file
file_handler = logging.FileHandler('/your-drive/titanic/model_training.log')
file_handler.setLevel(logging.INFO)
formatter = logging.Formatter('%(asctime)s - %(levelname)s - %(message)s')
file_handler.setFormatter(formatter)
logger.addHandler(file_handler)


In [6]:
# Load data
train_df = pd.read_csv('/your-drive/titanic/train.csv')
test_df = pd.read_csv('/your-drive/titanic/test.csv')
gender_submission = pd.read_csv('/your-drive/titanic/gender_submission.csv')

train_df.head()


,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S


In [7]:
train_df.info()
train_df.describe()
train_df.isnull().sum()


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 891 entries, 0 to 890
Data columns (total 12 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   PassengerId  891 non-null    int64  
 1   Survived     891 non-null    int64  
 2   Pclass       891 non-null    int64  
 3   Name         891 non-null    object 
 4   Sex          891 non-null    object 
 5   Age          714 non-null    float64
 6   SibSp        891 non-null    int64  
 7   Parch        891 non-null    int64  
 8   Ticket       891 non-null    object 
 9   Fare         891 non-null    float64
 10  Cabin        204 non-null    object 
 11  Embarked     889 non-null    object 
dtypes: float64(2), int64(5), object(5)
memory usage: 83.7+ KB


,0
PassengerId,0
Survived,0
Pclass,0
Name,0
Sex,0
Age,177
SibSp,0
Parch,0
Ticket,0
Fare,0


In [8]:
# Feature engineering functions
def get_title(name):
    title_search = re.search(' ([A-Za-z]+)\.', name)
    if title_search:
        return title_search.group(1)
    return ""

def add_features(data, is_train=True):
    # Extract Title from Name
    data['Title'] = data['Name'].apply(get_title)
    # Group rare titles together
    data['Title'] = data['Title'].replace(['Lady', 'Countess', 'Capt', 'Col', 'Don', 'Dr', 'Major',
                                           'Rev', 'Sir', 'Jonkheer', 'Dona'], 'Rare')
    data['Title'] = data['Title'].replace(['Mlle', 'Ms'], 'Miss')
    data['Title'] = data['Title'].replace('Mme', 'Mrs')

    # Family Size and IsAlone features
    data['FamilySize'] = data['SibSp'] + data['Parch'] + 1
    data['IsAlone'] = (data['FamilySize'] == 1).astype(int)

    # Cabin Deck
    data['Deck'] = data['Cabin'].apply(lambda x: x[0] if pd.notna(x) else 'M')

    # Fare per person
    data['FarePerPerson'] = data['Fare'] / data['FamilySize']

    # Log transformation of Fare
    data['LogFare'] = np.log1p(data['Fare'])

    # Bin Age into groups
    data['AgeGroup'] = pd.cut(data['Age'], bins=[0, 12, 18, 35, 60, 120], labels=['Child', 'Teen', 'YoungAdult', 'Adult', 'Senior'])

    # Interaction feature: Pclass and Sex
    data['Pclass*Sex'] = data['Pclass'].astype(str) + "_" + data['Sex']

    # Interaction feature: Pclass and FamilySize
    data['Pclass*FamilySize'] = data['Pclass'].astype(str) + "_" + data['FamilySize'].astype(str)

    # Drop unnecessary columns
    data.drop(['Name', 'Ticket', 'Cabin'], axis=1, inplace=True)

    return data

# Apply feature engineering
train_df = add_features(train_df, is_train=True)
test_df = add_features(test_df, is_train=False)


In [9]:
# Fill missing values
train_df['Age'] = train_df.groupby(['Pclass', 'Sex'])['Age'].transform(lambda x: x.fillna(x.median()))
test_df['Age'] = test_df.groupby(['Pclass', 'Sex'])['Age'].transform(lambda x: x.fillna(x.median()))
train_df['Embarked'].fillna(train_df['Embarked'].mode()[0], inplace=True)
test_df['Fare'].fillna(test_df['Fare'].median(), inplace=True)


In [12]:
# Verify Title Extraction
print(train_df['Title'].value_counts())

# Verify FamilySize and IsAlone
print(train_df[['SibSp', 'Parch', 'FamilySize', 'IsAlone']].head(10))

# Verify FarePerPerson and LogFare
print(train_df[['Fare', 'FamilySize', 'FarePerPerson', 'LogFare']].head(10))

# Verify AgeGroup Binning
print(train_df[['Age', 'AgeGroup']].head(10))
print(train_df['AgeGroup'].value_counts())

# Verify Pclass*Sex Interaction Feature
print(train_df[['Pclass', 'Sex', 'Pclass*Sex']].head(10))


Title
Mr        517
Miss      185
Mrs       126
Master     40
Rare       23
Name: count, dtype: int64
   SibSp  Parch  FamilySize  IsAlone
0      1      0           2        0
1      1      0           2        0
2      0      0           1        1
3      1      0           2        0
4      0      0           1        1
5      0      0           1        1
6      0      0           1        1
7      3      1           5        0
8      0      2           3        0
9      1      0           2        0
      Fare  FamilySize  FarePerPerson   LogFare
0   7.2500           2        3.62500  2.110213
1  71.2833           2       35.64165  4.280593
2   7.9250           1        7.92500  2.188856
3  53.1000           2       26.55000  3.990834
4   8.0500           1        8.05000  2.202765
5   8.4583           1        8.45830  2.246893
6  51.8625           1       51.86250  3.967694
7  21.0750           5        4.21500  3.094446
8  11.1333           3        3.71110  2.495954
9  30.0708 

In [13]:
# Define feature columns and target variable
X_train = train_df.drop(['Survived', 'PassengerId'], axis=1)
y_train = train_df['Survived']
X_test = test_df.drop(['PassengerId'], axis=1)

# Verify columns before preprocessing
print("X_train columns:", X_train.columns)
print("X_test columns:", X_test.columns)


X_train columns: Index(['Pclass', 'Sex', 'Age', 'SibSp', 'Parch', 'Fare', 'Embarked', 'Title',
       'FamilySize', 'IsAlone', 'Deck', 'FarePerPerson', 'LogFare', 'AgeGroup',
       'Pclass*Sex', 'Pclass*FamilySize'],
      dtype='object')
X_test columns: Index(['Pclass', 'Sex', 'Age', 'SibSp', 'Parch', 'Fare', 'Embarked', 'Title',
       'FamilySize', 'IsAlone', 'Deck', 'FarePerPerson', 'LogFare', 'AgeGroup',
       'Pclass*Sex', 'Pclass*FamilySize'],
      dtype='object')


In [14]:
# Update categorical features list after feature engineering
categorical_features = ['Sex', 'Embarked', 'Title', 'AgeGroup', 'Deck', 'Pclass*Sex']

# Numerical features to be scaled
numerical_features = X_train.select_dtypes(include=['int64', 'float64']).columns.tolist()

# Verify Feature Lists
print("Numerical Features:", numerical_features)
print("Categorical Features:", categorical_features)

# Define preprocessing pipeline
preprocessor = ColumnTransformer(
    transformers=[
        ('num', Pipeline(steps=[
            ('imputer', SimpleImputer(strategy='median')),  # Imputation first
            ('scaler', StandardScaler())  # Then scaling
        ]), numerical_features),
        ('cat', Pipeline(steps=[
            ('imputer', SimpleImputer(strategy='most_frequent')),  # Imputation for categorical features
            ('onehot', OneHotEncoder(drop='first', handle_unknown='ignore'))  # One-hot encoding
        ]), categorical_features)
    ])


# Fit the preprocessor on the training set
preprocessor.fit(X_train)


Numerical Features: ['Pclass', 'Age', 'SibSp', 'Parch', 'Fare', 'FamilySize', 'IsAlone', 'FarePerPerson', 'LogFare']
Categorical Features: ['Sex', 'Embarked', 'Title', 'AgeGroup', 'Deck', 'Pclass*Sex']


ColumnTransformer(transformers=[('num',
                                 Pipeline(steps=[('imputer',
                                                  SimpleImputer(strategy='median')),
                                                 ('scaler', StandardScaler())]),
                                 ['Pclass', 'Age', 'SibSp', 'Parch', 'Fare',
                                  'FamilySize', 'IsAlone', 'FarePerPerson',
                                  'LogFare']),
                                ('cat',
                                 Pipeline(steps=[('imputer',
                                                  SimpleImputer(strategy='most_frequent')),
                                                 ('onehot',
                                                  OneHotEncoder(drop='first',
                                                                handle_unknown='ignore'))]),
                                 ['Sex', 'Embarked', 'Title', 'AgeGroup',
                                  'Deck', 'Pclass*Sex'])])

In [15]:
# Preprocess the training and test sets
X_train_preprocessed = preprocessor.transform(X_train)
X_test_preprocessed = preprocessor.transform(X_test)

# Convert preprocessed data back to DataFrame
X_train_preprocessed_df = pd.DataFrame(X_train_preprocessed, columns=preprocessor.get_feature_names_out())
X_test_preprocessed_df = pd.DataFrame(X_test_preprocessed, columns=preprocessor.get_feature_names_out())

# Check the transformed data
print(X_train_preprocessed_df.head())
print(X_test_preprocessed_df.head())


   num__Pclass  num__Age  num__SibSp  num__Parch  num__Fare  num__FamilySize  \
0     0.827377 -0.534891    0.432793   -0.473674  -0.502445         0.059160   
1    -1.566107  0.668392    0.432793   -0.473674   0.786845         0.059160   
2     0.827377 -0.234070   -0.474545   -0.473674  -0.488854        -0.560975   
3    -1.566107  0.442776    0.432793   -0.473674   0.420730         0.059160   
4     0.827377  0.442776   -0.474545   -0.473674  -0.486337        -0.560975   

   num__IsAlone  num__FarePerPerson  num__LogFare  cat__Sex_male  ...  \
0     -1.231645           -0.454798     -0.879741            1.0  ...   
1     -1.231645            0.438994      1.361220            0.0  ...   
2      0.811922           -0.334757     -0.798540            0.0  ...   
3     -1.231645            0.185187      1.062038            0.0  ...   
4      0.811922           -0.331267     -0.784179            1.0  ...   

   cat__Deck_E  cat__Deck_F  cat__Deck_G  cat__Deck_M  cat__Deck_T  \
0         

In [16]:
# Initialize models with default parameters
log_reg = LogisticRegression(class_weight='balanced', random_state=42)
rf = RandomForestClassifier(class_weight='balanced', random_state=42)
gb = GradientBoostingClassifier(random_state=42)
xgb_model = XGBClassifier(use_label_encoder=False, eval_metric='logloss', random_state=42)
catboost_model = CatBoostClassifier(verbose=0, random_state=42)


In [17]:
# List of models to evaluate
models = {
    'LogisticRegression': log_reg,
    'RandomForest': rf,
    'GradientBoosting': gb,
    'XGBoost': xgb_model,
    'CatBoost': catboost_model
}

# Evaluate each model using cross-validation
for model_name, model in models.items():
    cv_scores = cross_val_score(model, X_train_preprocessed_df, y_train, cv=StratifiedKFold(n_splits=5), scoring='accuracy')
    print(f"{model_name} Cross-validation Accuracy: {cv_scores.mean():.4f} (+/- {cv_scores.std():.4f})")


LogisticRegression Cross-validation Accuracy: 0.8182 (+/- 0.0280)
RandomForest Cross-validation Accuracy: 0.8013 (+/- 0.0321)
GradientBoosting Cross-validation Accuracy: 0.8249 (+/- 0.0244)
XGBoost Cross-validation Accuracy: 0.8271 (+/- 0.0248)
CatBoost Cross-validation Accuracy: 0.8373 (+/- 0.0303)


In [18]:
param_distributions_rf = {
    'n_estimators': [100, 200, 300, 400, 500],
    'max_depth': [4, 8, 12, 16, None]
}

param_distributions_xgb = {
    'max_depth': [3, 5, 7, 9],
    'learning_rate': np.logspace(-3, 0, 10),
    'n_estimators': [100, 200, 300, 400]
}

param_distributions_catboost = {
    'depth': [4, 6, 8, 10],
    'learning_rate': np.logspace(-3, 0, 10),
    'n_estimators': [100, 200, 300, 400]
}

# RandomizedSearchCV for RandomForest
rf_search = RandomizedSearchCV(
    RandomForestClassifier(random_state=42),
    param_distributions=param_distributions_rf,
    n_iter=50,
    cv=StratifiedKFold(n_splits=5),
    scoring="accuracy",
    n_jobs=-1,
    random_state=42
)
rf_search.fit(X_train_preprocessed_df, y_train)
print(f"Best RandomForest model: {rf_search.best_params_}")
print(f"Best RandomForest score: {rf_search.best_score_:.4f}")

# RandomizedSearchCV for XGBoost
xgboost_search = RandomizedSearchCV(
    XGBClassifier(use_label_encoder=False, eval_metric='logloss', random_state=42),
    param_distributions=param_distributions_xgb,
    n_iter=50,
    cv=StratifiedKFold(n_splits=5),
    scoring="accuracy",
    n_jobs=-1,
    random_state=42
)
xgboost_search.fit(X_train_preprocessed_df, y_train)
print(f"Best XGBoost model: {xgboost_search.best_params_}")
print(f"Best XGBoost score: {xgboost_search.best_score_:.4f}")

# RandomizedSearchCV for CatBoost
catboost_search = RandomizedSearchCV(
    CatBoostClassifier(verbose=0, random_state=42),
    param_distributions=param_distributions_catboost,
    n_iter=50,
    cv=StratifiedKFold(n_splits=5),
    scoring="accuracy",
    n_jobs=-1,
    random_state=42
)
catboost_search.fit(X_train_preprocessed_df, y_train)
print(f"Best CatBoost model: {catboost_search.best_params_}")
print(f"Best CatBoost score: {catboost_search.best_score_:.4f}")


Best RandomForest model: {'n_estimators': 500, 'max_depth': 8}
Best RandomForest score: 0.8339
Best XGBoost model: {'n_estimators': 300, 'max_depth': 7, 'learning_rate': 0.004641588833612777}
Best XGBoost score: 0.8373
Best CatBoost model: {'n_estimators': 100, 'learning_rate': 0.1, 'depth': 8}
Best CatBoost score: 0.8395


In [19]:
stacking_clf_catboost = StackingClassifier(
    estimators=[('rf', rf_search.best_estimator_),
                ('xgb', xgboost_search.best_estimator_),
                ('cat', catboost_search.best_estimator_)],
    final_estimator=CatBoostClassifier(
        n_estimators=300,
        learning_rate=0.021544346900318832,
        depth=8,
        verbose=0,
        random_state=42
    )
)

# Train the CatBoost-based stacking classifier
stacking_clf_catboost.fit(X_train_preprocessed_df, y_train)

stacking_clf_mlp = StackingClassifier(
    estimators=[('rf', rf_search.best_estimator_),
                ('xgb', xgboost_search.best_estimator_),
                ('cat', catboost_search.best_estimator_)],
    final_estimator=MLPClassifier(hidden_layer_sizes=(50,), max_iter=500, random_state=42)
)

# Train the MLP-based stacking classifier
stacking_clf_mlp.fit(X_train_preprocessed_df, y_train)

stacking_clf_log = StackingClassifier(
    estimators=[('rf', rf_search.best_estimator_),
                ('cat', catboost_search.best_estimator_),
                ('xgb', xgboost_search.best_estimator_)],
    final_estimator=LogisticRegression()
)

# Train the Logistic_Regression-based stacking classifier
stacking_clf_log.fit(X_train_preprocessed_df, y_train)


StackingClassifier(estimators=[('rf',
                                RandomForestClassifier(max_depth=8,
                                                       n_estimators=500,
                                                       random_state=42)),
                               ('cat',
                                <catboost.core.CatBoostClassifier object at 0x7a2ee69a61d0>),
                               ('xgb',
                                XGBClassifier(base_score=None, booster=None,
                                              callbacks=None,
                                              colsample_bylevel=None,
                                              colsample_bynode=None,
                                              colsample_bytree=None,
                                              device=None,
                                              early_stopping_rounds=None,
                                              enable_...
                                              interaction_constraints=None,
                                              learning_rate=0.004641588833612777,
                                              max_bin=None,
                                              max_cat_threshold=None,
                                              max_cat_to_onehot=None,
                                              max_delta_step=None, max_depth=7,
                                              max_leaves=None,
                                              min_child_weight=None,
                                              missing=nan,
                                              monotone_constraints=None,
                                              multi_strategy=None,
                                              n_estimators=300, n_jobs=None,
                                              num_parallel_tree=None,
                                              random_state=42, ...))],
                   final_estimator=LogisticRegression())

In [21]:
# Cross-validation setup
cv = StratifiedKFold(n_splits=12, shuffle=True, random_state=42)

# Evaluate the CatBoost-based stacking classifier
scores_catboost = cross_val_score(stacking_clf_catboost, X_train_preprocessed_df, y_train, cv=cv, scoring='accuracy')
print(f"CatBoost StackingClassifier Cross-validation Accuracy: {scores_catboost.mean():.4f} (+/- {scores_catboost.std():.4f})")

# Evaluate the MLP-based stacking classifier
scores_mlp = cross_val_score(stacking_clf_mlp, X_train_preprocessed_df, y_train, cv=cv, scoring='accuracy')
print(f"MLP StackingClassifier Cross-validation Accuracy: {scores_mlp.mean():.4f} (+/- {scores_mlp.std():.4f})")

# Compare the cross-validation results
if scores_catboost.mean() > scores_mlp.mean():
    best_stacking_clf = stacking_clf_catboost
    print("CatBoost-based stacking classifier selected.")
else:
    best_stacking_clf = stacking_clf_mlp
    print("MLP-based stacking classifier selected.")

# Train the selected stacking classifier on the full training set
best_stacking_clf.fit(X_train_preprocessed_df, y_train)

# Predict on the test set using the best stacking model
predictions = best_stacking_clf.predict(X_test_preprocessed_df)

# Create final submission file
submission = pd.DataFrame({
    "PassengerId": test_df['PassengerId'],
    "Survived": predictions
})
submission.to_csv('/your-drive/titanic/final_submission.csv', index=False)


CatBoost StackingClassifier Cross-validation Accuracy: 0.8338 (+/- 0.0424)
MLP StackingClassifier Cross-validation Accuracy: 0.8361 (+/- 0.0354)
MLP-based stacking classifier selected.
